In [1]:
import cirq
import random


In [2]:

def prepare_entangled_pair():
    """Create an entangled Bell state (|Φ+⟩)."""
    qubit_a = cirq.NamedQubit("Alice")
    qubit_b = cirq.NamedQubit("Bob")
    circuit = cirq.Circuit()
    circuit.append(cirq.H(qubit_a))  # Apply Hadamard to Alice's qubit
    circuit.append(cirq.CNOT(qubit_a, qubit_b))  # Entangle qubits
    return qubit_a, qubit_b, circuit


In [3]:
def measure_in_basis(circuit, qubit, basis, key_name):
    """Add measurement in the specified basis to the circuit."""
    if basis == 'X':  # For X-basis, apply Hadamard before measuring
        circuit.append(cirq.H(qubit))
    circuit.append(cirq.measure(qubit, key=key_name))


In [4]:

def e91_protocol(num_bits):
    alice_bases = []
    bob_bases = []
    alice_results = []
    bob_results = []
    shared_key = []

    simulator = cirq.Simulator()

    for _ in range(num_bits):
        # Step 1: Prepare entangled pair
        qubit_a, qubit_b, circuit = prepare_entangled_pair()

        # Step 2: Randomly choose measurement bases for Alice and Bob
        alice_basis = random.choice(['Z', 'X'])
        bob_basis = random.choice(['Z', 'X'])

        alice_bases.append(alice_basis)
        bob_bases.append(bob_basis)

        # Step 3: Add measurements to the circuit
        measure_in_basis(circuit, qubit_a, alice_basis, "alice")
        measure_in_basis(circuit, qubit_b, bob_basis, "bob")

        # Step 4: Simulate the circuit
        results = simulator.run(circuit)
        alice_result = int(results.measurements['alice'][0][0])
        bob_result = int(results.measurements['bob'][0][0])

        alice_results.append(alice_result)
        bob_results.append(bob_result)

        # Step 5: Key sifting - only keep results when Alice and Bob chose the same basis
        if alice_basis == bob_basis:
            shared_key.append(alice_result)

    return alice_bases, bob_bases, alice_results, bob_results, shared_key


In [5]:
num_bits = 10
alice_bases, bob_bases, alice_results, bob_results, shared_key = e91_protocol(num_bits)


In [6]:

print("Alice's bases:    ", alice_bases)
print("Bob's bases:      ", bob_bases)
print("Alice's results:  ", alice_results)
print("Bob's results:    ", bob_results)
print("Shared key:       ", shared_key)

Alice's bases:     ['X', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'X']
Bob's bases:       ['Z', 'X', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'X']
Alice's results:   [1, 1, 1, 1, 1, 1, 0, 0, 1, 1]
Bob's results:     [0, 0, 1, 1, 1, 1, 1, 0, 1, 1]
Shared key:        [1, 0, 1, 1]
